In [1]:
from langchain_classic.chains.sequential import SimpleSequentialChain

print("hello")


hello


In [21]:
import os
import json
import pandas as pd
import traceback

In [22]:
from langchain_groq import ChatGroq

In [23]:
from dotenv import load_dotenv
load_dotenv()

True

In [24]:
KEY = os.getenv("api_keys")

In [35]:
llm = ChatGroq(
    temperature=0.5,
    groq_api_key= KEY,
    model_name="llama-3.3-70b-versatile",
    verbose = False
)

In [36]:
llm

ChatGroq(verbose=False, client=<groq.resources.chat.completions.Completions object at 0x0000012FA20880A0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000012FA2088700>, model_name='llama-3.3-70b-versatile', temperature=0.5, groq_api_key=SecretStr('**********'))

In [28]:
# !pip install langchain.prompts
# !pip uninstall -y langchain langchain-core langchain-community langchain-groq


In [29]:
# !pip install langchain==0.0.352
# !pip install langchain-community==0.0.20
# !pip install langchain-groq==0.0.1


In [43]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [31]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [32]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}


In [33]:
quiz_generation_prompt= PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template = TEMPLATE
)

In [34]:
quiz_chain = LLMChain(llm = llm , prompt= quiz_generation_prompt, output_key = "quiz", verbose = True)

In [38]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis.
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [39]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)


In [40]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)


In [44]:
generate_evaluate_chain=SequentialChain(
    chains=[quiz_chain,review_chain],
    input_variables=["text", "number", "subject", "tone", "response_json"],
    output_variables=["quiz", "review"],
    verbose=True,)


In [45]:
file_path= r"C:\Users\HP\PycharmProjects\Generative_AI\data.txt"

In [48]:
file_path

'C:\\Users\\HP\\PycharmProjects\\Generative_AI\\data.txt'

In [49]:
with open (file_path, 'r', encoding= "utf-8") as file:
     TEXT = file.read()

In [51]:
#TEXT

In [55]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [56]:
NUMBER=4
SUBJECT="Advantages and Disadvantages of Machine Learning and Deep Learning"
TONE="simple"

In [60]:
from langchain.globals import set_debug
set_debug(False)
from langchain.globals import set_llm_cache
set_llm_cache(None)



In [61]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine Learning (ML) and Deep Learning (DL) are two core branches of Artificial Intelligence (AI) that focus on enabling computers to learn from data. While both are used to make predictions and automate decision-making, they differ in how they process data and the complexity of models they use. Understanding the differences between them helps us to choose the right approach for a problem, optimize resources and achieving better results in real-world applications.

Machine Learning
Machine Learning is a branch of Artificial Intelligence that enables computer systems to learn patterns from data and make predictions or decisions without being explicitly programmed by humans.

Works with smaller datasets.
Requires manual feature extraction.
Easier to interpret and implement.
Best for structured data like tables or CSV files.
Types of Machine Learning Algorithms
Different types of M

In [62]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")


Total Tokens:2775
Prompt Tokens:2178
Completion Tokens:597
Total Cost:0.0


In [64]:
response

{'text': 'Machine Learning (ML) and Deep Learning (DL) are two core branches of Artificial Intelligence (AI) that focus on enabling computers to learn from data. While both are used to make predictions and automate decision-making, they differ in how they process data and the complexity of models they use. Understanding the differences between them helps us to choose the right approach for a problem, optimize resources and achieving better results in real-world applications.\n\nMachine Learning\nMachine Learning is a branch of Artificial Intelligence that enables computer systems to learn patterns from data and make predictions or decisions without being explicitly programmed by humans.\n\nWorks with smaller datasets.\nRequires manual feature extraction.\nEasier to interpret and implement.\nBest for structured data like tables or CSV files.\nTypes of Machine Learning Algorithms\nDifferent types of ML Algorithms:\n\nSupervised Learning: Here model learns from labelled datasets, where th

In [65]:
quiz = response.get("quiz")

In [66]:
quiz

'{"1": {"mcq": "What is a key characteristic of Machine Learning?", "options": {"a": "It requires large datasets to function", "b": "It can only be used for image recognition", "c": "It can learn patterns from data without being explicitly programmed", "d": "It is a type of Deep Learning"}, "correct": "c"}, \n"2": {"mcq": "What type of data is Deep Learning best suited for?", "options": {"a": "Structured data like tables or CSV files", "b": "Unstructured data such as images, audio or text", "c": "Only text data", "d": "Only image data"}, "correct": "b"}, \n"3": {"mcq": "What is a disadvantage of Deep Learning?", "options": {"a": "It is easy to interpret", "b": "It can run on CPUs", "c": "It is difficult to interpret and requires more computation power", "d": "It works well with small datasets"}, "correct": "c"}, \n"4": {"mcq": "When should you use Machine Learning over Deep Learning?", "options": {"a": "When working with large unstructured datasets", "b": "When data is structured and l

In [67]:
quiz= json.loads(quiz)

In [68]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [69]:
quiz_table_data

[{'MCQ': 'What is a key characteristic of Machine Learning?',
  'Choices': 'a: It requires large datasets to function | b: It can only be used for image recognition | c: It can learn patterns from data without being explicitly programmed | d: It is a type of Deep Learning',
  'Correct': 'c'},
 {'MCQ': 'What type of data is Deep Learning best suited for?',
  'Choices': 'a: Structured data like tables or CSV files | b: Unstructured data such as images, audio or text | c: Only text data | d: Only image data',
  'Correct': 'b'},
 {'MCQ': 'What is a disadvantage of Deep Learning?',
  'Choices': 'a: It is easy to interpret | b: It can run on CPUs | c: It is difficult to interpret and requires more computation power | d: It works well with small datasets',
  'Correct': 'c'},
 {'MCQ': 'When should you use Machine Learning over Deep Learning?',
  'Choices': 'a: When working with large unstructured datasets | b: When data is structured and limited, and interpretability is important | c: When you

In [70]:
quiz=pd.DataFrame(quiz_table_data)


In [71]:
quiz

,MCQ,Choices,Correct
0,What is a key characteristic of Machine Learning?,a: It requires large datasets to function | b:...,c
1,What type of data is Deep Learning best suited...,a: Structured data like tables or CSV files | ...,b
2,What is a disadvantage of Deep Learning?,a: It is easy to interpret | b: It can run on ...,c
3,When should you use Machine Learning over Deep...,a: When working with large unstructured datase...,b


In [72]:
quiz.to_csv("machinelearning.csv",index=False)

In [73]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'01_01_2026_01_40_27'